# Validation GPU Phase 4.3 - WENO5 + SSP-RK3 GPU

**Objectif** : Valider l'intégration complète WENO5 + SSP-RK3 sur GPU

- ✅ Simulation CPU WENO5 + SSP-RK3 de référence
- ✅ Simulation GPU WENO5 + SSP-RK3  
- ✅ Export des résultats dans `output_gpu_phase43.zip`

## Instructions Kaggle

1. **Activer GPU** dans les paramètres du notebook
2. **Exécuter toutes les cellules** (Runtime → Run All)
3. **Télécharger** `output_gpu_phase43.zip` depuis l'onglet Output
4. **Extraire localement** et utiliser les scripts d'analyse adaptés

📋 Test final de l'intégration WENO5 (ordre 5 spatial) + SSP-RK3 (ordre 3 temporel) sur GPU.

## 1. Setup Environnement Kaggle

In [51]:
!rm -rf Code-traffic-flow
#!/usr/bin/env python3

# Cloner le dépôt et setup
!git clone https://github.com/elonmj/Code-traffic-flow.git

import os
import sys
import numpy as np
from datetime import datetime

# --- Nouvelle Approche : Travailler Directement dans le Dossier Cloné ---
# Se déplacer dans le dossier du projet cloné
project_dir = "Code-traffic-flow"
os.chdir(project_dir)
print(f"✅ Répertoire de travail changé pour: {os.getcwd()}")

# Ajouter le répertoire de travail au path Python
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

print(f"🐍 Python path configuré: {sys.path[:3]}...")  # Afficher les 3 premiers éléments

# Vérification des fichiers critiques
critical_files = [
    'code/simulation/runner.py',
    'config/scenario_weno5_ssprk3_gpu_validation.yml'
]

print("\n🔍 Vérification des fichiers critiques...")
for file_path in critical_files:
    if os.path.exists(file_path):
        print(f"   ✅ {file_path} trouvé")
    else:
        print(f"   ❌ {file_path} manquant")

print("\n🎯 Setup terminé - prêt pour les simulations")

Cloning into 'Code-traffic-flow'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 164 (delta 35), reused 152 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (164/164), 737.32 KiB | 18.43 MiB/s, done.
Resolving deltas: 100% (35/35), done.
✅ Répertoire de travail changé pour: /kaggle/working/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow
🐍 Python path configuré: ['/kaggle/working/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow', '/kaggle/working/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow/Code-traffic-flow', '/kag

## 2. Vérification CUDA

In [52]:
# Vérification CUDA
try:
    from numba import cuda
    if cuda.is_available():
        device = cuda.get_current_device()
        print(f"✅ GPU: {device.name}")
        print(f"   Capacité: {device.compute_capability}")
    else:
        print("❌ CUDA non disponible")
except ImportError:
    print("❌ Numba CUDA non trouvé")

✅ GPU: b'Tesla T4'
   Capacité: (7, 5)


## 3. Simulation CPU WENO5 + SSP-RK3 (Référence)

In [53]:
# Simulation CPU de référence avec WENO5 + SSP-RK3
print("🖥️ SIMULATION CPU WENO5 + SSP-RK3 (référence)")
print("="*60)

import time

# Approche pragmatique : utiliser directement main_simulation.py
print("🔬 Test du modèle ARZ complet avec WENO5 + SSP-RK3 sur CPU")
print("Scénario: WENO5 + SSP-RK3 validation")

# Lancer une simulation CPU avec WENO5 + SSP-RK3
start_time = time.time()

!python code/main_simulation.py \
    --scenario config/scenario_weno5_ssprk3_gpu_validation.yml \
    --device cpu \
    --output_dir results_cpu_weno5_ssprk3

end_time = time.time()
duration_cpu = end_time - start_time

print(f"\n⏱️ Durée simulation CPU: {duration_cpu:.2f} secondes")
print("✅ Simulation CPU WENO5 + SSP-RK3 terminée")

# Vérifier les résultats
if os.path.exists("results_cpu_weno5_ssprk3"):
    print("📁 Résultats CPU générés avec succès")
    cpu_success = True
else:
    print("❌ Aucun résultat CPU trouvé")
    cpu_success = False

🖥️ SIMULATION CPU WENO5 + SSP-RK3 (référence)
🔬 Test du modèle ARZ complet avec WENO5 + SSP-RK3 sur CPU
Scénario: WENO5 + SSP-RK3 validation
--------------------------------------------------
Starting Full Simulation
Scenario Config: config/scenario_weno5_ssprk3_gpu_validation.yml
Base Config:     config/config_base.yml
Output Dir:      results_cpu_weno5_ssprk3
--------------------------------------------------
Initializing simulation from scenario: config/scenario_weno5_ssprk3_gpu_validation.yml
Using device: cpu
DEBUG PARAMS: Reading K_m_kmh = 5.0
DEBUG PARAMS: Reading K_c_kmh = 7.5
DEBUG PARAMS: Assigned self.K_m = 1.3888888888888888
DEBUG PARAMS: Assigned self.K_c = 2.0833333333333335
Parameters loaded for scenario: scenario_weno5_ssprk3_gpu_validation
Grid initialized: Grid1D(N=200, xmin=0.0, xmax=1000.0, dx=5.0000, ghost=3, N_total=206, R loaded=No)
  Loading road quality type: uniform
  Uniform road quality value: 1
Road quality loaded.
Initial state created.
Initial boundary co

## 4. Simulation GPU WENO5 + SSP-RK3 (Test)

In [54]:
# Simulation GPU avec WENO5 + SSP-RK3
print("🚀 SIMULATION GPU WENO5 + SSP-RK3")
print("="*50)

if cpu_success:
    print("🔬 Test du modèle ARZ complet avec WENO5 + SSP-RK3 sur GPU")
    print("Scénario: WENO5 + SSP-RK3 validation")
    
    # Lancer une simulation GPU avec WENO5 + SSP-RK3
    start_time = time.time()
    
    !python code/main_simulation.py \
        --scenario config/scenario_weno5_ssprk3_gpu_validation.yml \
        --device gpu \
        --output_dir results_gpu_weno5_ssprk3
    
    end_time = time.time()
    duration_gpu = end_time - start_time
    
    print(f"\n⏱️ Durée simulation GPU: {duration_gpu:.2f} secondes")
    print("✅ Simulation GPU WENO5 + SSP-RK3 terminée")
    
    # Speedup
    if duration_cpu > 0:
        speedup = duration_cpu / duration_gpu
        print(f"🚀 Speedup: {speedup:.2f}x")
    
    # Vérifier les résultats
    if os.path.exists("results_gpu_weno5_ssprk3"):
        print("📁 Résultats GPU générés avec succès")
        gpu_success = True
    else:
        print("❌ Aucun résultat GPU trouvé")
        gpu_success = False
        
else:
    print("⚠️ Simulation GPU ignorée (échec CPU)")
    gpu_success = False

🚀 SIMULATION GPU WENO5 + SSP-RK3
🔬 Test du modèle ARZ complet avec WENO5 + SSP-RK3 sur GPU
Scénario: WENO5 + SSP-RK3 validation
--------------------------------------------------
Starting Full Simulation
Scenario Config: config/scenario_weno5_ssprk3_gpu_validation.yml
Base Config:     config/config_base.yml
Output Dir:      results_gpu_weno5_ssprk3
--------------------------------------------------
Initializing simulation from scenario: config/scenario_weno5_ssprk3_gpu_validation.yml
Using device: gpu
DEBUG PARAMS: Reading K_m_kmh = 5.0
DEBUG PARAMS: Reading K_c_kmh = 7.5
DEBUG PARAMS: Assigned self.K_m = 1.3888888888888888
DEBUG PARAMS: Assigned self.K_c = 2.0833333333333335
Parameters loaded for scenario: scenario_weno5_ssprk3_gpu_validation
Grid initialized: Grid1D(N=200, xmin=0.0, xmax=1000.0, dx=5.0000, ghost=3, N_total=206, R loaded=No)
  Loading road quality type: uniform
  Uniform road quality value: 1
Road quality loaded.
Initial state created.
Transferring initial state and r

## 4.1. GPU Fix - Retry After Boundary Conditions Fix

## 5. Export des Résultats Phase 4.3

In [ ]:
# GPU Retry après fix syncthreads
print("🔧 GPU RETRY APRÈS FIX SYNCTHREADS")
print("="*50)

if cpu_success:
    print("🚀 Test WENO5 + SSP-RK3 GPU après correction")
    
    # Lancer une simulation GPU avec WENO5 + SSP-RK3 (retry)
    start_time = time.time()
    
    !python code/main_simulation.py \
        --scenario config/scenario_weno5_ssprk3_gpu_validation.yml \
        --device gpu \
        --output_dir results_gpu_weno5_ssprk3_retry
    
    end_time = time.time()
    duration_gpu_retry = end_time - start_time
    
    print(f"\n⏱️ Durée simulation GPU (retry): {duration_gpu_retry:.2f} secondes")
    
    # Speedup
    if duration_cpu > 0:
        speedup_retry = duration_cpu / duration_gpu_retry
        print(f"🚀 Speedup (retry): {speedup_retry:.2f}x")
    
    # Vérifier les résultats
    if os.path.exists("results_gpu_weno5_ssprk3_retry"):
        print("📁 Résultats GPU (retry) générés avec succès")
        print("✅ GPU WENO5 + SSP-RK3 VALIDATION RÉUSSIE!")
        gpu_success = True
        duration_gpu = duration_gpu_retry  # Mettre à jour pour l'export
    else:
        print("❌ Aucun résultat GPU (retry) trouvé")
        
else:
    print("⚠️ Simulation GPU ignorée (échec CPU)")

In [55]:
# Export des résultats pour analyse locale Phase 4.3
import zipfile
import json
import glob

print("📦 EXPORT DES RÉSULTATS PHASE 4.3")
print("="*40)

# Créer dossier output principal
output_dir = "output_gpu_phase43"
os.makedirs(output_dir, exist_ok=True)

# Timestamp pour les fichiers
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

files_exported = []

# Collecter les résultats CPU et GPU depuis les dossiers générés
cpu_results_dir = "results_cpu_weno5_ssprk3"
gpu_results_dir = "results_gpu_weno5_ssprk3"

# Copier les résultats CPU
if cpu_success and os.path.exists(cpu_results_dir):
    cpu_files = glob.glob(f"{cpu_results_dir}/*")
    for file_path in cpu_files:
        if os.path.isfile(file_path):
            filename = os.path.basename(file_path)
            new_filename = f"cpu_{filename}"
            dest_path = os.path.join(output_dir, new_filename)
            import shutil
            shutil.copy2(file_path, dest_path)
            files_exported.append(dest_path)
    print(f"✅ CPU WENO5 + SSP-RK3: {len([f for f in cpu_files if os.path.isfile(f)])} fichiers copiés")

# Copier les résultats GPU
if gpu_success and os.path.exists(gpu_results_dir):
    gpu_files = glob.glob(f"{gpu_results_dir}/*")
    for file_path in gpu_files:
        if os.path.isfile(file_path):
            filename = os.path.basename(file_path)
            new_filename = f"gpu_{filename}"
            dest_path = os.path.join(output_dir, new_filename)
            shutil.copy2(file_path, dest_path)
            files_exported.append(dest_path)
    print(f"✅ GPU WENO5 + SSP-RK3: {len([f for f in gpu_files if os.path.isfile(f)])} fichiers copiés")

# Métadonnées de la validation Phase 4.3
metadata = {
    "timestamp": timestamp,
    "phase": "4.3",
    "objective": "Validation WENO5 + SSP-RK3 GPU vs CPU",
    "approach": "Direct main_simulation.py execution",
    "spatial_scheme": "weno5",
    "time_scheme": "ssprk3",
    "cpu_success": cpu_success,
    "gpu_success": gpu_success,
    "cpu_duration": duration_cpu if cpu_success else None,
    "gpu_duration": duration_gpu if gpu_success else None,
    "speedup": duration_cpu/duration_gpu if cpu_success and gpu_success else None,
    "spatial_order": 5,
    "temporal_order": 3,
    "scenario_file": "scenario_weno5_ssprk3_gpu_validation.yml"
}

metadata_file = f"{output_dir}/validation_metadata_phase43_{timestamp}.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)
files_exported.append(metadata_file)
print(f"✅ Métadonnées: {metadata_file}")

# Informations de configuration Phase 4.3
config_info = {
    "scenario": "scenario_weno5_ssprk3_gpu_validation.yml",
    "execution_method": "main_simulation.py with command line args",
    "phase": "4.3",
    "focus": "WENO5 + SSP-RK3 integration validation",
    "orders": {"spatial": 5, "temporal": 3},
    "expected_benefits": ["High spatial accuracy", "Strong stability preserving", "GPU acceleration"],
    "output_directories": {
        "cpu": "results_cpu_weno5_ssprk3",
        "gpu": "results_gpu_weno5_ssprk3"
    }
}

config_file = f"{output_dir}/config_info_phase43_{timestamp}.json"
with open(config_file, 'w') as f:
    json.dump(config_info, f, indent=2)
files_exported.append(config_file)
print(f"✅ Config: {config_file}")

print(f"\n📁 Fichiers dans {output_dir}:")
for file in files_exported:
    if os.path.exists(file):
        size = os.path.getsize(file) / 1024  # KB
        print(f"   {os.path.basename(file)} ({size:.1f} KB)")

# Statut final simple
comparison_success = cpu_success and gpu_success
status = "SUCCESS" if comparison_success else "PARTIAL" if cpu_success or gpu_success else "FAILED"

📦 EXPORT DES RÉSULTATS PHASE 4.3
✅ CPU WENO5 + SSP-RK3: 0 fichiers copiés
✅ Métadonnées: output_gpu_phase43/validation_metadata_phase43_20250714_182308.json
✅ Config: output_gpu_phase43/config_info_phase43_20250714_182308.json

📁 Fichiers dans output_gpu_phase43:
   validation_metadata_phase43_20250714_182308.json (0.4 KB)
   config_info_phase43_20250714_182308.json (0.5 KB)


## 6. Création ZIP Final Phase 4.3

In [56]:
# Créer le ZIP final Phase 4.3
# IMPORTANT: Créer le ZIP dans /kaggle/working pour qu'il soit téléchargeable
zip_filename = "../output_gpu_phase43.zip"  # Remonter d'un niveau vers /kaggle/working

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in files_exported:
        if os.path.exists(file):
            zipf.write(file, os.path.basename(file))

zip_size = os.path.getsize(zip_filename) / (1024*1024)  # MB

print(f"📦 ZIP créé: {zip_filename} ({zip_size:.2f} MB)")
print(f"   📍 Localisation: /kaggle/working/output_gpu_phase43.zip")
print(f"\n📥 INSTRUCTIONS:")
print(f"1. Télécharger output_gpu_phase43.zip depuis l'onglet Output")
print(f"2. Extraire en local")
print(f"3. Utiliser les scripts d'analyse locaux adaptés pour Phase 4.3")

# Résumé final Phase 4.3
print(f"\n🎯 RÉSUMÉ VALIDATION PHASE 4.3")
print(f"="*45)
print(f"Objectif: Validation WENO5 + SSP-RK3 sur GPU")
print(f"Méthode: Exécution directe main_simulation.py")
print(f"Schémas: weno5 (ordre 5) + ssprk3 (ordre 3)")
print(f"Simulation CPU: {'✅' if cpu_success else '❌'}")
print(f"Simulation GPU: {'✅' if gpu_success else '❌'}")
if cpu_success and gpu_success:
    print(f"Speedup GPU: {duration_cpu/duration_gpu:.2f}x")
print(f"Statut: {status}")
print(f"Export: ✅ output_gpu_phase43.zip")

print(f"\n🎉 VALIDATION PHASE 4.3 TERMINÉE")
print(f"   WENO5 + SSP-RK3 GPU: {'✅ TESTS EXÉCUTÉS' if gpu_success else '❌ PROBLÈME DÉTECTÉ'}")
print(f"   Approche simplifiée avec main_simulation.py ✅")

📦 ZIP créé: ../output_gpu_phase43.zip (0.00 MB)
   📍 Localisation: /kaggle/working/output_gpu_phase43.zip

📥 INSTRUCTIONS:
1. Télécharger output_gpu_phase43.zip depuis l'onglet Output
2. Extraire en local
3. Utiliser les scripts d'analyse locaux adaptés pour Phase 4.3

🎯 RÉSUMÉ VALIDATION PHASE 4.3
Objectif: Validation WENO5 + SSP-RK3 sur GPU
Méthode: Exécution directe main_simulation.py
Schémas: weno5 (ordre 5) + ssprk3 (ordre 3)
Simulation CPU: ✅
Simulation GPU: ❌
Statut: PARTIAL
Export: ✅ output_gpu_phase43.zip

🎉 VALIDATION PHASE 4.3 TERMINÉE
   WENO5 + SSP-RK3 GPU: ❌ PROBLÈME DÉTECTÉ
   Approche simplifiée avec main_simulation.py ✅
